In [5]:
import os
import cv2
import time
import json
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

In [6]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''
var = json.load(open("credidentials.json"))
subscription_key= var["API_KEY"]
endpoint = var["END_POINT"]
client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [7]:
anotari_manuale = {
    "bike01.jpg": [(5, 32, 411, 404)],
    "bike02.jpg": [(15, 88, 381, 320)],
    "bike03.jpg": [(159, 148, 345, 410)],
    "bike04.jpg": [(2, 2, 417, 418)],
    "bike05.jpg": [(69, 54, 357, 350)],
    "bike06.jpg": [(69, 150, 197, 398)],
    "bike07.jpg": [(60, 184, 298, 420)],
    "bike08.jpg": [(55, 8, 389, 352)],
    "bike09.jpg": [(5, 12, 382, 386)],
    "bike10.jpg": [(142, 126, 375, 410)]
}
cale_folder = "bikes"
lista_imagini = ["bike{:02d}.jpg".format(i) for i in range(1, 11)] + ["traffic{:02d}.jpg".format(i) for i in range(1, 11)]

In [8]:
def clasificare_imagine(cale_imagine):
    """
    Utilizează serviciul Azure pentru a analiza imaginea și a determina dacă aceasta conține o bicicletă.
    Returnează True dacă apare termenul 'bicycle' în descriere, altfel False.
    """
    with open(cale_imagine, "rb") as imagine:
        analiza = client.describe_image_in_stream(imagine)
    prezice_bicicleta = False
    if analiza.captions:
        #for cap in analiza.captions:
        #    print(f"    - '{cap.text}' (încredere: {cap.confidence:.2f})")
        for descriere in analiza.captions:
            if "bicycle" in descriere.text.lower():
                prezice_bicicleta = True
                break
    return prezice_bicicleta, analiza

In [9]:
def testeaza_clasificarea(lista_imagini, cale_folder):
    """
    Parcurge imaginile și determină dacă predicția corespunde etichetei reale (nume fișier).
    Returnează un dicționar cu rezultate și acuratețea totală.
    """
    rezultate = {}
    for imagine in lista_imagini:
        eticheta_real = imagine.startswith("bike")
        cale_completa = os.path.join(cale_folder, imagine)
        predictie, analiza = clasificare_imagine(cale_completa)
        rezultate[imagine] = {"predictie": predictie, "eticheta": eticheta_real, "detalii": analiza}
        print(f"{imagine}: predictie = {predictie}, eticheta reala = {eticheta_real}")

    corecte = sum(1 for img in lista_imagini if rezultate[img]["predictie"] == rezultate[img]["eticheta"])
    acuratete = corecte / len(lista_imagini)
    print(f"\nAcuratete clasificare: {acuratete:.2f}")
    return rezultate, acuratete


In [10]:
print("\n 1: Clasificare imagini")
rezultate_clasificare, acuratete_finala = testeaza_clasificarea(lista_imagini, cale_folder)


 1: Clasificare imagini
bike01.jpg: predictie = True, eticheta reala = True
bike02.jpg: predictie = True, eticheta reala = True
bike03.jpg: predictie = True, eticheta reala = True
bike04.jpg: predictie = False, eticheta reala = True
bike05.jpg: predictie = True, eticheta reala = True
bike06.jpg: predictie = True, eticheta reala = True
bike07.jpg: predictie = True, eticheta reala = True
bike08.jpg: predictie = True, eticheta reala = True
bike09.jpg: predictie = True, eticheta reala = True
bike10.jpg: predictie = True, eticheta reala = True
traffic01.jpg: predictie = False, eticheta reala = False
traffic02.jpg: predictie = False, eticheta reala = False
traffic03.jpg: predictie = False, eticheta reala = False
traffic04.jpg: predictie = False, eticheta reala = False
traffic05.jpg: predictie = False, eticheta reala = False
traffic06.jpg: predictie = False, eticheta reala = False
traffic07.jpg: predictie = False, eticheta reala = False
traffic08.jpg: predictie = True, eticheta reala = False

In [58]:
def detectare_biciclete(cale_imagine, prag_incredere=0.3):
    """
    Folosește AI Azure pentru a detecta automat obiectele relevante (biciclete).
    Returnează o listă de chenare (x, y, w, h).
    """
    with open(cale_imagine, "rb") as imagine:
        detectii = client.detect_objects_in_stream(imagine)
    chenar_gasit = []
    if detectii.objects:
        for obiect in detectii.objects:
            #print(f"[OBIECT] '{obiect.object_property}' detectat cu scor {obiect.confidence:.2f}")
            eticheta = obiect.object_property.lower()
            if ("bicycle" in eticheta or "bike" in eticheta) and obiect.confidence >= prag_incredere:
                coord = obiect.rectangle
                chenar_gasit.append((coord.x, coord.y, coord.w, coord.h))
    return chenar_gasit

In [59]:
def deseneaza_chenare(cale_imagine, chenare, cale_salveaza, culoare=(0, 255, 0)):
    """
    Desenează chenare automate pe imagine și salvează rezultatul.
    """
    img = cv2.imread(cale_imagine)
    for (x, y, w, h) in chenare:
        cv2.rectangle(img, (x, y), (x + w, y + h), culoare, 2)
    cv2.imwrite(cale_salveaza, img)

In [60]:
print("\n 2a: Detectare automată biciclete")
rezultate_detectie = {}
for imagine in lista_imagini:
    if imagine.startswith("bike"):
        cale = os.path.join(cale_folder, imagine)
        chenare_gasite = detectare_biciclete(cale, prag_incredere=0.3)
        rezultate_detectie[imagine] = chenare_gasite
        print(f"{imagine}: {len(chenare_gasite)} detectii -> {chenare_gasite}")
        if chenare_gasite:
            cale_salvare = os.path.join(cale_folder, "auto_" + imagine)
            deseneaza_chenare(cale, chenare_gasite, cale_salvare)
        time.sleep(3)


 2a: Detectare automată biciclete
[OBIECT] 'bicycle' detectat cu scor 0.94
bike01.jpg: 1 detectii -> [(3, 16, 409, 394)]
[OBIECT] 'Wheel' detectat cu scor 0.52
[OBIECT] 'bicycle' detectat cu scor 0.91
bike02.jpg: 1 detectii -> [(9, 90, 357, 231)]
[OBIECT] 'car' detectat cu scor 0.58
[OBIECT] 'person' detectat cu scor 0.79
[OBIECT] 'person' detectat cu scor 0.88
[OBIECT] 'bicycle' detectat cu scor 0.80
bike03.jpg: 1 detectii -> [(155, 153, 183, 252)]
[OBIECT] 'bicycle' detectat cu scor 0.82
bike04.jpg: 1 detectii -> [(0, 2, 414, 410)]
[OBIECT] 'bicycle' detectat cu scor 0.75
bike05.jpg: 1 detectii -> [(66, 36, 283, 299)]
[OBIECT] 'car' detectat cu scor 0.63
[OBIECT] 'person' detectat cu scor 0.73
[OBIECT] 'person' detectat cu scor 0.80
[OBIECT] 'cycle' detectat cu scor 0.59
bike06.jpg: 0 detectii -> []
[OBIECT] 'person' detectat cu scor 0.80
[OBIECT] 'bicycle' detectat cu scor 0.86
bike07.jpg: 1 detectii -> [(51, 206, 257, 210)]
[OBIECT] 'person' detectat cu scor 0.92
bike08.jpg: 0 det

In [40]:
def deseneaza_chenare_manual(cale_imagine, chenare_manuale, cale_salveaza):
    """
    Desenează chenare realizate manual pe imagine, cu culoare roșie.
    """
    img = cv2.imread(cale_imagine)
    for (x1, y1, x2, y2) in chenare_manuale:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.imwrite(cale_salveaza, img)

In [48]:
print("\n 2b: Etichetare manuală")
for imagine, etichete in anotari_manuale.items():
    cale = os.path.join(cale_folder, imagine)
    cale_salvare = os.path.join(cale_folder, "manual_" + imagine)
    deseneaza_chenare_manual(cale, etichete, cale_salvare)
    print(f"Salvat: {imagine}")


 2b: Etichetare manuală
Salvat: bike01.jpg
Salvat: bike02.jpg
Salvat: bike03.jpg
Salvat: bike04.jpg
Salvat: bike05.jpg
Salvat: bike06.jpg
Salvat: bike07.jpg
Salvat: bike08.jpg
Salvat: bike09.jpg
Salvat: bike10.jpg


In [61]:
def transforma_format_chenar(box):
    """
    Converteste formatul (x, y, w, h) în (x1, y1, x2, y2)
    """
    x, y, w, h = box
    return (x, y, x + w, y + h)

In [62]:
def calculeaza_iou(boxA, boxB):
    """
    Calculează Intersection over Union pentru două chenare (format x1, y1, x2, y2).
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    lat_inter = max(0, xB - xA)
    inaltime_inter = max(0, yB - yA)
    aria_inter = lat_inter * inaltime_inter

    aria_A = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    aria_B = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = aria_inter / float(aria_A + aria_B - aria_inter) if (aria_A + aria_B - aria_inter) > 0 else 0
    return iou


In [63]:
def evaluare_detectie(anotari_manuale, detectii_ai, prag_iou=0.5):
    """
    Evaluează detectările făcute de AI comparativ cu etichetele manuale.
    Returnează IoU mediu, precizie și recall.
    """
    scoruri_iou = []
    total_tp = 0
    total_fp = 0
    total_fn = 0

    for imagine, etichete in anotari_manuale.items():
        detectate = detectii_ai.get(imagine, [])
        etichete_corelate = [False] * len(etichete)
        tp_local = 0

        for chenar_detectat in detectate:
            chenar_convertit = transforma_format_chenar(chenar_detectat)
            cel_mai_bun_iou = 0
            idx_bun = -1
            for i, et_manual in enumerate(etichete):
                iou = calculeaza_iou(chenar_convertit, et_manual)
                if iou > cel_mai_bun_iou:
                    cel_mai_bun_iou = iou
                    idx_bun = i
            if cel_mai_bun_iou >= prag_iou and idx_bun != -1 and not etichete_corelate[idx_bun]:
                tp_local += 1
                etichete_corelate[idx_bun] = True
                scoruri_iou.append(cel_mai_bun_iou)
            else:
                total_fp += 1

        fn_local = len(etichete) - tp_local
        #print(f"[IMAGINE] {imagine}: TP = {tp_local}, FP = {len(detectate) - tp_local}, FN = {fn_local}")
        total_fn += fn_local
        total_tp += tp_local

    iou_mediu = sum(scoruri_iou) / len(scoruri_iou) if scoruri_iou else 0
    precizie = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0

    print(f"\nEvaluare finala:")
    print(f"IoU mediu: {iou_mediu:.2f}")
    print(f"Precizie: {precizie:.2f}")
    print(f"Recall: {recall:.2f}")

    return {"iou": iou_mediu, "precizie": precizie, "recall": recall}

In [64]:
print("\n 2c: Evaluare performanță detectie")
metrice = evaluare_detectie(anotari_manuale, rezultate_detectie, prag_iou=0.5)


 2c: Evaluare performanță detectie
[IMAGINE] bike01.jpg: TP = 1, FP = 0, FN = 0
[IMAGINE] bike02.jpg: TP = 1, FP = 0, FN = 0
[IMAGINE] bike03.jpg: TP = 1, FP = 0, FN = 0
[IMAGINE] bike04.jpg: TP = 1, FP = 0, FN = 0
[IMAGINE] bike05.jpg: TP = 1, FP = 0, FN = 0
[IMAGINE] bike06.jpg: TP = 0, FP = 0, FN = 1
[IMAGINE] bike07.jpg: TP = 1, FP = 0, FN = 0
[IMAGINE] bike08.jpg: TP = 0, FP = 0, FN = 1
[IMAGINE] bike09.jpg: TP = 1, FP = 0, FN = 0
[IMAGINE] bike10.jpg: TP = 1, FP = 0, FN = 0

Evaluare finala:
IoU mediu: 0.91
Precizie: 1.00
Recall: 0.80
